In [1]:
#import Dependencies
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
import pymongo 
import requests
from config import api_key

In [2]:
#executable path for window users
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
#Executable path for Mac Users
# executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
# browser = Browser('chrome', **executable_path, headless=False)

In [4]:
#url of page to be scraped from indeed
url = 'https://www.indeed.com/jobs?q=Software+Engineer&l=Irvine%2C+CA&radius=100'
browser.visit(url)

In [5]:
#BeautifulSoup object Parser
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [6]:
#result to return the job title
titles = soup.find_all("h2", class_ = "title")
print(titles)
#title[1].get_text()

[<h2 class="title">
<a class="jobtitle turnstileLink" data-tn-element="jobTitle" href="/pagead/clk?mo=r&amp;ad=-6NYlbfkN0DSJVOgFNjTH3NHsz1y5W7iEiQ8XFPauIDjgJYsd-OdNLeg3X3B2bDiJFBMXKrgB8Cc3lh2uHFnxwFa3_G81uJq8h0YPsu1bqktj9WwY8gX8ghU6g2YHgticn1Z9tA2oy4mIx5BC5HjhypjwhLn92E6PsRN5qwF-IqablSFGNvWbLk1k6eaNp-15lpsCwhqE1GzMCUhyw_-UyMUm2Yl349KuFxXN_xMKVOtwk01zoHSYtWKJx4XoDERinZCmd0XHLUcshEJ67Apgy_wTtFZ29NQEqMkC3_SfYgpvMwkX-1sNEcfS2D9O-NY9Ny6GW8nSfgDVPqBEjyLx73o1IaGDWGsctgmYDcmGQ-nR8YfiMYKPfmG94atUvWPhFqYd43QHIOfX3zYzUCdjFaX22rcRMajcHui9R2uPLDIxnxtMBA7TBBMdmW9FSCX0lVOTZggm8M7Twi4EAOfuTCVvUUYSyek&amp;p=0&amp;fvj=1&amp;vjs=3" id="sja0" onclick="setRefineByCookie(['radius']); sjoc('sja0', 1); convCtr('SJ'); rclk(this,jobmap[0],true,1);" onmousedown="sjomd('sja0'); clk('sja0'); rclk(this,jobmap[0],1);" rel="noopener nofollow" target="_blank" title="Software Engineer">
<b>Software</b> <b>Engineer</b></a>
</h2>, <h2 class="title">
<a class="jobtitle turnstileLink" data-tn-element="jobTitle" href="/page

In [7]:
#result to be Returned as an iterable list
for title in titles:
    job_title = title.get_text()
    print(job_title)
    



Software Engineer



Sr. Software Test Engineer II
new


Software Engineer



Software Engineer- Entry Level



Software Engineer, University Grad
new


Software Developer



Software Engineer
new


Software Engineer for Game Control Systems and Prototyping
new


Jr Full Stack Software Engineer



Software Developer



Software Engineer



Software Development Engineer
new


Software Engineer 2, Front End
new


Junior Software Engineer
new


Software Engineer (Remote)
new


In [8]:

try:
    dismiss = browser.find_by_xpath('/html/body/div[2]/div/section/div/div[2]')
    more_jobs = browser.find_by_xpath('/html/body/table[2]/tbody/tr/td/table/tbody/tr/td[1]/nav/div/ul/li[6]/a/span')
    no_thanks = browser.find_by_xpath('/html/body/div[3]/div[2]/div[4]/div[3]/div[2]/a')
    dismiss.first.click()
    more_jobs.first.click()
    no_thanks.first.click()
except:
    print("not found")

not found


In [9]:
#Empty list to append the the result and index 0 is an int 
job_titles = [0]
 #loop through the result
for x in range(1, 2):
    titles = soup.find_all("h2", class_ = "title")
    for title in titles:
        job_titles.append(title.text)
        print('page:', x, '-------------')
        print(title.text)
        browser.find_by_xpath('/html/body/table[2]/tbody/tr/td/table/tbody/tr/td[1]/nav/div/ul/li[6]/a/span/span')


page: 1 -------------


Software Engineer

page: 1 -------------


Sr. Software Test Engineer II
new
page: 1 -------------


Software Engineer

page: 1 -------------


Software Engineer- Entry Level

page: 1 -------------


Software Engineer, University Grad
new
page: 1 -------------


Software Developer

page: 1 -------------


Software Engineer
new
page: 1 -------------


Software Engineer for Game Control Systems and Prototyping
new
page: 1 -------------


Jr Full Stack Software Engineer

page: 1 -------------


Software Developer

page: 1 -------------


Software Engineer

page: 1 -------------


Software Development Engineer
new
page: 1 -------------


Software Engineer 2, Front End
new
page: 1 -------------


Junior Software Engineer
new
page: 1 -------------


Software Engineer (Remote)
new


In [10]:
#Use replace to print out a clean job title
clean_job_titles = []

#loop through the result
for job in job_titles[1:]:
    clean_job_title = job.replace('\n', '')
    clean_job_titles.append(clean_job_title)

In [11]:
#print clean job titles to check
print(clean_job_titles)

['Software Engineer', 'Sr. Software Test Engineer IInew', 'Software Engineer', 'Software Engineer- Entry Level', 'Software Engineer, University Gradnew', 'Software Developer', 'Software Engineernew', 'Software Engineer for Game Control Systems and Prototypingnew', 'Jr Full Stack Software Engineer', 'Software Developer', 'Software Engineer', 'Software Development Engineernew', 'Software Engineer 2, Front Endnew', 'Junior Software Engineernew', 'Software Engineer (Remote)new']


In [12]:
#result to return the job locations
locations = soup.find_all("span", class_ = "location accessible-contrast-color-location")
locations

[<span class="location accessible-contrast-color-location">Irvine, CA</span>,
 <span class="location accessible-contrast-color-location">Irvine, CA</span>,
 <span class="location accessible-contrast-color-location">Los Angeles, CA</span>,
 <span class="location accessible-contrast-color-location">Irvine, CA 92614 <span style="font-size: smaller">(Business District area)</span></span>,
 <span class="location accessible-contrast-color-location">Burbank, CA</span>,
 <span class="location accessible-contrast-color-location">Newport Beach, CA 92660</span>,
 <span class="location accessible-contrast-color-location">Irvine, CA</span>,
 <span class="location accessible-contrast-color-location">Irvine, CA</span>,
 <span class="location accessible-contrast-color-location">Los Angeles, CA 90071 <span style="font-size: smaller">(Downtown area)</span></span>,
 <span class="location accessible-contrast-color-location">La Puente, CA 91744</span>]

In [13]:
#Empty list to append the result
job_locations = []

#loop through the returned result
for x in range(1, 2):
    locations = soup.find_all("span", class_ = "location accessible-contrast-color-location")
    for location in locations:
        job_locations.append(location.text)
        print('page:', x, '-------------')
        print(location.text)
        browser.find_by_xpath('/html/body/table[2]/tbody/tr/td/table/tbody/tr/td[1]/div[7]/div[1]/span[1]')
        

page: 1 -------------
Irvine, CA
page: 1 -------------
Irvine, CA
page: 1 -------------
Los Angeles, CA
page: 1 -------------
Irvine, CA 92614 (Business District area)
page: 1 -------------
Burbank, CA
page: 1 -------------
Newport Beach, CA 92660
page: 1 -------------
Irvine, CA
page: 1 -------------
Irvine, CA
page: 1 -------------
Los Angeles, CA 90071 (Downtown area)
page: 1 -------------
La Puente, CA 91744


In [14]:
#print job locations
print(job_locations)

['Irvine, CA', 'Irvine, CA', 'Los Angeles, CA', 'Irvine, CA 92614 (Business District area)', 'Burbank, CA', 'Newport Beach, CA 92660', 'Irvine, CA', 'Irvine, CA', 'Los Angeles, CA 90071 (Downtown area)', 'La Puente, CA 91744']


In [15]:
#zip both results into a list
new_list = list(zip(clean_job_titles, job_locations))

In [16]:
#print new_list that has both the job_titiles and locations
print(new_list)

[('Software Engineer', 'Irvine, CA'), ('Sr. Software Test Engineer IInew', 'Irvine, CA'), ('Software Engineer', 'Los Angeles, CA'), ('Software Engineer- Entry Level', 'Irvine, CA 92614 (Business District area)'), ('Software Engineer, University Gradnew', 'Burbank, CA'), ('Software Developer', 'Newport Beach, CA 92660'), ('Software Engineernew', 'Irvine, CA'), ('Software Engineer for Game Control Systems and Prototypingnew', 'Irvine, CA'), ('Jr Full Stack Software Engineer', 'Los Angeles, CA 90071 (Downtown area)'), ('Software Developer', 'La Puente, CA 91744')]


In [17]:
#append list into a python dictionary
updated_list = dict(new_list)
updated_list

{'Software Engineer': 'Los Angeles, CA',
 'Sr. Software Test Engineer IInew': 'Irvine, CA',
 'Software Engineer- Entry Level': 'Irvine, CA 92614 (Business District area)',
 'Software Engineer, University Gradnew': 'Burbank, CA',
 'Software Developer': 'La Puente, CA 91744',
 'Software Engineernew': 'Irvine, CA',
 'Software Engineer for Game Control Systems and Prototypingnew': 'Irvine, CA',
 'Jr Full Stack Software Engineer': 'Los Angeles, CA 90071 (Downtown area)'}

In [18]:
#Initialize pymongo to work with MongoDBs
#CONNECT MONGO DB
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [19]:
#convert newlist to pandas dataframe
updated_list_df = pd.DataFrame(new_list)
updated_list_df

,0,1
0,Software Engineer,"Irvine, CA"
1,Sr. Software Test Engineer IInew,"Irvine, CA"
2,Software Engineer,"Los Angeles, CA"
3,Software Engineer- Entry Level,"Irvine, CA 92614 (Business District area)"
4,"Software Engineer, University Gradnew","Burbank, CA"
5,Software Developer,"Newport Beach, CA 92660"
6,Software Engineernew,"Irvine, CA"
7,Software Engineer for Game Control Systems and...,"Irvine, CA"
8,Jr Full Stack Software Engineer,"Los Angeles, CA 90071 (Downtown area)"
9,Software Developer,"La Puente, CA 91744"


In [20]:
#append colums keys
New_job_location_df = pd.DataFrame(new_list, columns=['Job_Title', 'Location'])
New_job_location_df

,Job_Title,Location
0,Software Engineer,"Irvine, CA"
1,Sr. Software Test Engineer IInew,"Irvine, CA"
2,Software Engineer,"Los Angeles, CA"
3,Software Engineer- Entry Level,"Irvine, CA 92614 (Business District area)"
4,"Software Engineer, University Gradnew","Burbank, CA"
5,Software Developer,"Newport Beach, CA 92660"
6,Software Engineernew,"Irvine, CA"
7,Software Engineer for Game Control Systems and...,"Irvine, CA"
8,Jr Full Stack Software Engineer,"Los Angeles, CA 90071 (Downtown area)"
9,Software Developer,"La Puente, CA 91744"


In [21]:
#Define database and collection
#Database = Indeed_db
#Collections = Software_jobs
client.Indeed_db.software_jobs.insert_many(New_job_location_df.to_dict('records'))

In [ ]:
#Done